# Classical Control - Code Laboratory

**Section 3: Create Circuits - Part 5** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

### Conditional Operations (⚠️ EXAM CRITICAL!)
| Method | Syntax | Use When |
|--------|--------|----------|
| `if_test()` | `with qc.if_test((clbit, value)):` | Modern: conditional block |
| `if_test()` with else | `with qc.if_test(...) as else_:` | If-else branching |
| `c_if()` | `gate.c_if(clbit, value)` | Deprecated: single gate |

### Classical Registers
| Method | Signature | Returns |
|--------|-----------|---------|
| `ClassicalRegister()` | `ClassicalRegister(size, name)` | `ClassicalRegister` |
| `qc.measure()` | `measure(qubit, clbit)` | `InstructionSet` |
| `qc.reset()` | `reset(qubit)` | `InstructionSet` |

### Classical Expression (expr module)
| Function | Purpose |
|----------|---------|
| `expr.logic_and(a, b)` | a AND b |
| `expr.logic_or(a, b)` | a OR b |
| `expr.logic_not(a)` | NOT a |
| `expr.equal(a, b)` | a == b |

### ⚠️ Register Value Encoding (BINARY!)
| Bit 1 | Bit 0 | Value |
|-------|-------|-------|
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 2 |
| 1 | 1 | 3 |

---

In [ ]:
"""
Qiskit Code Laboratory - Classical Control
===========================================
Prerequisites: See README.md for conceptual background
"""

# Standard imports
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector

# Utility functions for this notebook
def verify_state(qc, expected_state, tolerance=1e-10):
    """Verify circuit produces expected state."""
    actual = Statevector.from_instruction(qc)
    return np.allclose(actual.data, expected_state, atol=tolerance)

def show_circuit_info(qc, name="Circuit"):
    """Display circuit information."""
    print(f"{name}:")
    print(f"  Qubits: {qc.num_qubits}")
    print(f"  Classical bits: {qc.num_clbits}")
    print(f"  Depth: {qc.depth()}")

print("✅ Environment ready - Classical Control Lab")

---

## `if_test()` - Conditional Gate Execution

### Signature
```python
with QuantumCircuit.if_test((classical_bit_or_register, value)) as else_:
    # gates to execute if condition is true
with else_:
    # gates to execute if condition is false (optional)
```

### Parameters
| Parameter | Type | Description |
|-----------|------|-------------|
| `classical_bit_or_register` | `Clbit` or `ClassicalRegister` | What to check |
| `value` | `int` | Value to compare (0, 1 for bit; 0-3 for 2-bit register) |

---

### Basic Usage

In [ ]:
# ============================================================
# if_test() - BASIC USAGE
# ============================================================

qc = QuantumCircuit(2, 2)

# Measure qubit 0
qc.h(0)
qc.measure(0, 0)

# Apply X to qubit 1 IF classical bit 0 is 1
with qc.if_test((qc.clbits[0], 1)):
    qc.x(1)

print("Conditional X gate:")
print(qc.draw())

# Verify structure
assert qc.num_qubits == 2, "Should have 2 qubits"
assert qc.num_clbits == 2, "Should have 2 classical bits"
print("\n✅ X(1) executes ONLY if measurement result is 1!")

---

### Parameter Variations: Multiple Conditions

In [ ]:
# ============================================================
# if_test() - MULTIPLE CONDITIONS
# ============================================================

qc = QuantumCircuit(3, 3)

# Measure all qubits
qc.h(range(3))
qc.measure(range(3), range(3))

# Different conditions
with qc.if_test((qc.clbits[0], 1)):  # If bit 0 is 1
    qc.x(0)
with qc.if_test((qc.clbits[1], 0)):  # If bit 1 is 0
    qc.y(1)
with qc.if_test((qc.clbits[2], 1)):  # If bit 2 is 1
    qc.z(2)

print("Multiple conditions:")
print(qc.draw())
print("\n✅ Each if_test checks independently")

---

## Classical Registers - Conditional on Register Value

---

In [ ]:
# ============================================================
# CLASSICAL REGISTERS - Named Registers
# ============================================================

cr1 = ClassicalRegister(2, 'results')
cr2 = ClassicalRegister(1, 'flag')
qr = QuantumRegister(3, 'q')

qc = QuantumCircuit(qr, cr1, cr2)

# Measure to different registers
qc.h(range(3))
qc.measure(qr[0:2], cr1)
qc.measure(qr[2], cr2)

print("Multiple classical registers:")
print(qc.draw())
print(f"\nRegisters: {[r.name for r in qc.cregs]}")
print("✅ Named registers for organized output")

In [ ]:
# ============================================================
# CONDITIONAL ON ENTIRE REGISTER (⚠️ BINARY VALUE!)
# ============================================================

cr = ClassicalRegister(2, 'c')
qc = QuantumCircuit(QuantumRegister(3), cr)

qc.h([0, 1])
qc.measure([0, 1], cr)

# Check register value (0, 1, 2, or 3)
with qc.if_test((cr, 3)):  # Execute if cr == 3 (binary 11)
    qc.x(2)

print("Conditional on register value:")
print(qc.draw())

print("\n⚠️ Register value encoding (BINARY!):")
print("  cr=0: bits [0,0]")
print("  cr=1: bits [1,0]")
print("  cr=2: bits [0,1]")
print("  cr=3: bits [1,1]  ← this condition")

---

## 2. If-Else Structure with `as else_:`

In [ ]:
# ============================================================
# IF-ELSE STRUCTURE: 'as else_:' SYNTAX
# ============================================================

qc = QuantumCircuit(2, 1)

qc.h(0)
qc.measure(0, 0)

# Pattern: if_test() as else_  →  with else_:
with qc.if_test((qc.clbits[0], 1)) as else_:
    qc.x(1)      # If c[0] == 1, apply X
with else_:
    qc.z(1)      # Else (c[0] == 0), apply Z

print("If-Else Structure:")
print(qc.draw())

# Verification: circuit has conditional gates
assert qc.num_qubits == 2
assert qc.num_clbits == 1
print("\n✅ 'as else_:' captures the else branch!")

---

## 3. Classical Expressions with `qiskit.circuit.classical.expr`

Complex conditions (AND, OR, comparisons) require the `expr` module:

In [ ]:
# ============================================================
# CLASSICAL EXPRESSIONS: AND CONDITION
# ============================================================
from qiskit.circuit.classical import expr

cr = ClassicalRegister(2, 'c')
qc = QuantumCircuit(3, cr)

qc.h([0, 1])
qc.measure([0, 1], [0, 1])

# AND condition: execute if BOTH bits are 1
condition_and = expr.logic_and(cr[0], cr[1])
with qc.if_test(condition_and):
    qc.x(2)
    
print("Classical Expression (AND):")
print(qc.draw())

# Verification
assert qc.num_qubits == 3
print("\n✅ X gate applied only if c[0]=1 AND c[1]=1")

In [ ]:
# ============================================================
# EXPR MODULE: AVAILABLE FUNCTIONS REFERENCE
# ============================================================

print("expr Module Functions (EXAM!):")
print("=" * 50)
print()
print("LOGIC OPERATIONS:")
print("  expr.logic_and(a, b)   → a AND b")
print("  expr.logic_or(a, b)    → a OR b")
print("  expr.logic_not(a)      → NOT a")
print()
print("COMPARISON OPERATIONS:")
print("  expr.equal(a, b)       → a == b")
print("  expr.not_equal(a, b)   → a != b")
print("  expr.greater(a, b)     → a > b")
print("  expr.less(a, b)        → a < b")
print("  expr.greater_equal(a, b) → a >= b")
print("  expr.less_equal(a, b)  → a <= b")
print()
print("✅ Use expr for complex conditions with if_test()!")

---

## 4. ⚠️ TRAP DEMO: if_test() vs c_if() (EXAM!)

| Feature | c_if() (Deprecated) | if_test() (Modern) |
|---------|---------------------|-------------------|
| **Syntax** | `gate.c_if(bit, val)` | `with qc.if_test((bit, val)):` |
| **Else clause** | ❌ Not supported | ✅ `as else_:` |
| **Multiple gates** | One gate per call | Block of gates |
| **Complex conditions** | ❌ Limited | ✅ `expr` module |
| **Status** | ⚠️ Deprecated | ✅ Recommended |

# ============================================================
# ⚠️ TRAP DEMO: c_if() vs if_test() SYNTAX COMPARISON
# ============================================================

# --- c_if() (DEPRECATED but may appear on exam!) ---
qc_old = QuantumCircuit(2, 1)
qc_old.h(0)
qc_old.measure(0, 0)
# c_if() is a METHOD on the gate instruction
qc_old.x(1).c_if(qc_old.clbits[0], 1)  # One gate per call

print("c_if() - Deprecated Syntax:")
print(qc_old.draw())

# --- if_test() (MODERN) ---
qc_new = QuantumCircuit(2, 1)
qc_new.h(0)
qc_new.measure(0, 0)
# if_test() is a context manager on the circuit
with qc_new.if_test((qc_new.clbits[0], 1)):
    qc_new.x(1)  # Multiple gates allowed!
    qc_new.z(1)

print("\nif_test() - Modern Syntax:")
print(qc_new.draw())

print("\n⚠️ EXAM: Know BOTH syntaxes!")
print("  c_if():    gate.c_if(bit, value)  ← method on gate")
print("  if_test(): with qc.if_test((bit, value)):  ← context manager")

---

## 5. 🔴 QUANTUM TELEPORTATION with if_test() (GUARANTEED EXAM!)

In [ ]:
# ============================================================
# 🔴 QUANTUM TELEPORTATION - MEMORIZE THIS PATTERN!
# ============================================================

qc = QuantumCircuit(3, 2)

# STEP 0: Prepare state to teleport (qubit 0)
# Teleporting |+⟩ state
qc.h(0)
qc.barrier()

# STEP 1: Create entanglement (qubits 1 and 2)
qc.h(1)
qc.cx(1, 2)
qc.barrier()

# STEP 2: Bell measurement (qubits 0 and 1)
qc.cx(0, 1)
qc.h(0)
qc.barrier()
qc.measure([0, 1], [0, 1])
qc.barrier()

# STEP 3: Conditional corrections (CRITICAL!)
with qc.if_test((qc.clbits[1], 1)):  # If c[1]=1, apply X
    qc.x(2)
with qc.if_test((qc.clbits[0], 1)):  # If c[0]=1, apply Z
    qc.z(2)

print("QUANTUM TELEPORTATION:")
print(qc.draw())

# Verification
assert qc.num_qubits == 3, "Teleportation needs 3 qubits"
assert qc.num_clbits == 2, "Teleportation needs 2 classical bits"
print("\n✅ Pattern verified: 3 qubits, 2 classical bits")
print("🎯 EXAM: X correction → c[1], Z correction → c[0]")

### 🔥 TELEPORTATION EXAM FACTS (MEMORIZE!):

| Step | Operation | Qubits | Notes |
|------|-----------|--------|-------|
| 1. Entangle | H + CNOT | 1, 2 | Always same! |
| 2. Bell measure | CNOT + H + measure | 0, 1 | CNOT first, then H |
| 3. X correction | `if c[1]=1: X(2)` | 2 | Second measurement! |
| 4. Z correction | `if c[0]=1: Z(2)` | 2 | First measurement! |

**⚠️ TRAP**: X depends on c[1], NOT c[0]!

In [ ]:
# ============================================================
# ⚠️ TRAP DEMO: TELEPORTATION CORRECTION ORDER
# ============================================================

# WRONG order vs CORRECT order demo
print("⚠️ TRAP: Which bit controls which correction?")
print("=" * 50)
print()
print("Bell measurement measures: [q0, q1] → [c0, c1]")
print()
print("❌ WRONG (common mistake):")
print("   with qc.if_test((c[0], 1)): qc.x(2)  # X on c[0]?")
print("   with qc.if_test((c[1], 1)): qc.z(2)  # Z on c[1]?")
print()
print("✅ CORRECT:")
print("   with qc.if_test((c[1], 1)): qc.x(2)  # X on c[1]!")
print("   with qc.if_test((c[0], 1)): qc.z(2)  # Z on c[0]!")
print()
print("🎯 MNEMONIC: 'X-1, Z-0' (X uses c[1], Z uses c[0])")

---

## 6. Mid-Circuit Measurements

In [ ]:
# ============================================================
# MID-CIRCUIT MEASUREMENT: Measure Before Circuit Ends
# ============================================================

qc = QuantumCircuit(2, 2)

# First part
qc.h(0)
qc.cx(0, 1)

# Mid-circuit measurement - collapses q0!
qc.measure(0, 0)

# Continue AFTER measurement (q0 is now 0 or 1)
qc.h(1)
qc.measure(1, 1)

print("Mid-circuit measurement:")
print(qc.draw())

# Verification
assert qc.num_qubits == 2
assert qc.num_clbits == 2
print("\n✅ Measurement doesn't end circuit - operations continue!")
print("⚠️ After measure: qubit is classical (0 or 1), not superposition")

---

## 7. reset() - Reset and Reuse Qubits

In [ ]:
# ============================================================
# RESET: Force Qubit to |0⟩ and Reuse
# ============================================================

qc = QuantumCircuit(1, 2)

# First use
qc.h(0)
qc.measure(0, 0)

# Reset to |0⟩ - regardless of measured state!
qc.reset(0)

# Reuse qubit in fresh |0⟩ state
qc.x(0)
qc.measure(0, 1)

print("Reset and reuse:")
print(qc.draw())

# Verification
assert 'reset' in str(qc.data).lower() or any('reset' in str(instr).lower() for instr, _, _ in qc.data)
print("\n✅ reset() forces qubit to |0⟩!")
print("📌 Use case: Qubit recycling in long circuits")

---

## 🧪 CODE CHALLENGES

Complete these challenges to verify your understanding:

In [ ]:
# ============================================================
# CHALLENGE 1: Build Teleportation Circuit
# ============================================================
# Create a complete teleportation circuit for |1⟩ state

# Your solution:
qc = QuantumCircuit(3, 2)

# Prepare |1⟩ state on qubit 0
qc.x(0)
qc.barrier()

# Create entanglement (qubits 1 and 2)
qc.h(1)
qc.cx(1, 2)
qc.barrier()

# Bell measurement (qubits 0 and 1)
qc.cx(0, 1)
qc.h(0)
qc.measure([0, 1], [0, 1])
qc.barrier()

# Corrections
with qc.if_test((qc.clbits[1], 1)):
    qc.x(2)
with qc.if_test((qc.clbits[0], 1)):
    qc.z(2)

# ============================================================
# VERIFICATION
# ============================================================
assert qc.num_qubits == 3, "Need 3 qubits for teleportation"
assert qc.num_clbits == 2, "Need 2 classical bits"
print("✅ Challenge 1 PASSED: Teleportation circuit correct!")
print(qc.draw())

In [ ]:
# ============================================================
# CHALLENGE 2: If-Else Pattern
# ============================================================
# Create circuit that applies X if bit is 1, else applies Z

# Your solution:
qc = QuantumCircuit(2, 1)

qc.h(0)
qc.measure(0, 0)

# If-else structure
with qc.if_test((qc.clbits[0], 1)) as else_:
    qc.x(1)   # If c[0] == 1
with else_:
    qc.z(1)   # Else (c[0] == 0)

# ============================================================
# VERIFICATION
# ============================================================
assert qc.num_qubits == 2
assert qc.num_clbits == 1
print("✅ Challenge 2 PASSED: If-else pattern correct!")
print(qc.draw())

In [ ]:
# ============================================================
# CHALLENGE 3: Register Value Encoding
# ============================================================
# What integer value does a 2-bit register show if bit[1]=1, bit[0]=0?

# Your solution:
# bits [bit_1, bit_0] = [1, 0]
# Integer = bit_1 * 2^1 + bit_0 * 2^0
#         = 1 * 2 + 0 * 1 = 2

expected_value = 2

# ============================================================
# VERIFICATION
# ============================================================
# Binary encoding table
print("Register Value Encoding:")
print("  bits [0,0] = 0")
print("  bits [1,0] = 1")
print("  bits [0,1] = 2  ← bit_1=1, bit_0=0")
print("  bits [1,1] = 3")
print()
assert expected_value == 2, f"Expected 2, got {expected_value}"
print("✅ Challenge 3 PASSED: Value is 2!")

---

## 📚 Resources

- [Qiskit if_test Documentation](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qiskit.circuit.QuantumCircuit.if_test)
- [Classical Expressions (expr)](https://docs.quantum.ibm.com/api/qiskit/circuit_classical)
- [Dynamic Circuits Tutorial](https://learning.quantum.ibm.com/tutorial/dynamic-circuits)

---

**Section 3: Create Quantum Circuits** | [← README](./README.md) | [Next: Dynamic Circuits →](./dynamic_circuits.ipynb)

*Code Laboratory v1.0 | Qiskit 1.x Compatible*